# Example 02 - Sig53 Classifier
This notebook walks through a simple example of how to use the clean Sig53 dataset, load a pre-trained supported model, and evaluate the trained network's performance. Note that the experiment and the results herein are not to be interpreted with any significant value but rather serve simply as a practical example of how the `torchsig` dataset and tools can be used and integrated within a typical [PyTorch](https://pytorch.org/) and/or [PyTorch Lightning](https://www.pytorchlightning.ai/) workflow.

----

## Import Libraries
First, import all the necessary public libraries as well as a few classes from the `torchsig` toolkit.

In [ ]:
from torchsig.transforms.target_transforms import DescToClassIndex
from torchsig.utils.writer import DatasetCreator
from torchsig.transforms.transforms import (
    RandomPhaseShift,
    Normalize,
    ComplexTo2D,
    Compose,
)
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import LightningModule, Trainer
from sklearn.metrics import classification_report
from torchsig.utils.cm_plotter import plot_confusion_matrix
from torchsig.datasets.sig53 import Sig53
from torchsig.datasets.modulations import ModulationsDataset
from torchsig.datasets.datamodules import Sig53DataModule
from torch.nn import CrossEntropyLoss
from torchsig.datasets import conf
from torch import optim
import pytorch_lightning as pl
from tqdm import tqdm
import torch.nn.functional as F
from torchmetrics import Accuracy
import numpy as np
import torch

from torchsig.models import EfficientNet1d

----
### Instantiate Sig53 Dataset
Here, we instantiate the Sig53 clean training dataset and the Sig53 clean validation dataset. We demonstrate how to compose multiple TorchSig transforms together, using a data impairment with a random phase shift that uniformly samples a phase offset between -1 pi and +1 pi. The next transform normalizes the complex tensor, and the final transform converts the complex data to a real-valued tensor with the real and imaginary parts as two channels. We additionally provide a target transform that maps the `SignalMetadata` objects, that are part of `SignalData` objects, to a desired format for the model we will train. In this case, we use the `DescToClassIndex` target transform to map class names to their indices within an ordered class list. Finally, we sample from our datasets and print details in order to confirm functionality.

For more details on the Sig53 dataset instantiations, please see `00_example_sig53_dataset.ipynb`.

In [ ]:
# Specify Sig53 Options
root = "./datasets/wideband_sig53"
impaired = False
class_list = list(Sig53._idx_to_name_dict.values())
batch_size = 64
num_workers = 4

transform = Compose(
    [
        RandomPhaseShift(phase_offset=(-1, 1)),
        Normalize(norm=np.inf),
        ComplexTo2D(),
    ]
)
target_transform = DescToClassIndex(class_list=class_list)

datamodule = Sig53DataModule(
    root=root,
    impaired=False,
    transform=transform,
    target_transform=target_transform,
    batch_size=batch_size,
    num_workers=num_workers
)
datamodule.prepare_data()
datamodule.setup("fit")

# Retrieve a sample and print out information to verify
idx = np.random.randint(len(datamodule.train))
data, label = datamodule.train[idx]
print("Dataset length: {}".format(len(datamodule.train)))
print("Data shape: {}".format(data.shape))
print("Label Index: {}".format(label))
print("Label Class: {}".format(Sig53.convert_idx_to_name(label)))

----
## Instantiate Supported TorchSig Model
Below, we create a 1d EfficientNet-B0 model, and then conform it to a PyTorch LightningModule for training.

In [ ]:
model = EfficientNet1d(2,53)

In [ ]:
class ExampleClassifier(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.mdl = model
        self.loss_fn = CrossEntropyLoss()
        self.acc = Accuracy(task="multiclass",num_classes=53,top_k=5)
    def forward(self,batch):
        return self.mdl(batch[0].float()).argmax(-1),batch[1]

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
    
    def training_step(self, batch):
        x, y = (batch[0].float(),batch[1])
        y_hat = self.mdl(x)
        return self.loss_fn(y_hat, y)

    def validation_step(self, batch):
        x, y = (batch[0].float(),batch[1])
        y_hat = self.mdl(x)
        loss = self.loss_fn(y_hat, y)
        self.acc(y_hat,y)
        results = {"val/loss": loss, "val/acc": self.acc}
        self.log_dict(results, prog_bar=True)
        return results
example_model = ExampleClassifier(model)

----
## Train the Model
To train the model, we first create a `ModelCheckpoint` to monitor the validation loss over time and save the best model as we go. The network is then instantiated and passed into a `Trainer` to kick off training.

In [ ]:
trainer = pl.Trainer(
    max_epochs=25,
)
trainer.fit(example_model, datamodule)

In [ ]:
preds = trainer.predict(example_model,datamodule.val_dataloader())

In [ ]:
y_hat = torch.cat([p[0] for p in preds])
y = torch.cat([p[1] for p in preds])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

accuracy_per_class = []
for cls in range(53):
    # Select predictions and labels for the current class
    cls_predictions = y_hat[y == cls]
    cls_labels = y[y == cls]
    
    # Calculate accuracy
    accuracy = torch.mean((cls_predictions == cls_labels).float())
    accuracy_per_class.append(accuracy)

# Plot the accuracy per class
plt.figure(figsize=(20, 5))
plt.bar(class_list, accuracy_per_class)
plt.xlabel('Class')
plt.ylabel('Accuracy')
plt.title('Accuracy per Class')
plt.xticks(class_list,rotation=70)
plt.show()
